In [1]:
import pandas as pd
import numpy as np

# Data Loading 

In [2]:
train = pd.read_hdf('train.h5')

In [19]:
train[['technical_20', 'technical_20_acc']]

MemoryError: 

# Feature Columns 

In [4]:
excl = ['id', 'y', 'timesatmp']
origin_features = [feature for feature in train.columns if feature not in excl]

# Add Accumulation Features 

In [12]:
tec20_cum = train.technical_20.cumsum()

In [13]:
np.corrcoef(tec20_cum, train.y)[0][1]

0.0018214872539307387

In [14]:
train.fillna(train.mean(), inplace=True)
for feature in origin_features[:3]:
    train[feature + '_acc'] = train[feature].cumsum()

MemoryError: 